In [2]:
import requests
import csv

from datetime import datetime

def get_carbon_intensity(postcode, dateFrom, dateTo):
    #url = f'https://api.carbonintensity.org.uk/regional/intensity/{date}/pt24h/postcode/{postcode}'
    url = f'https://api.carbonintensity.org.uk/regional/intensity/{dateFrom}/{dateTo}/postcode/{postcode}'
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to retrieve data: {response.status_code}")
        return None

def find_min_max_intensity(data):
    if 'data' in data:
        extracted_data = []
        for entry in data['data']['data']:
            from_time = entry['from']
            to_time = entry['to']
            forecast = entry['intensity']['forecast']
            extracted_data.append([from_time, to_time, forecast])

    csv_file = '../data/carbon_intensity.csv'
    with open(csv_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['PeriodFrom', 'PeriodTo', 'CarbonIntensity'])
        writer.writerows(extracted_data)

    print(f'Data has been exported to {csv_file}')
    if extracted_data:
        min_intensity = min(extracted_data, key=lambda x: x[2])
        max_intensity = max(extracted_data, key=lambda x: x[2])
        return min_intensity, max_intensity
    else:
        return None, None

# Define the postcode and date
postcode = 'DE74'
dateFrom = '2024-06-01'  # Date in the format YYYY-MM-DD
dateTo = '2024-06-02'

# Fetch the data
carbon_intensity_data = get_carbon_intensity(postcode, dateFrom, dateTo)

if carbon_intensity_data:
    # Find the minimum and maximum intensity
    min_intensity, max_intensity = find_min_max_intensity(carbon_intensity_data)
    if min_intensity is not None and max_intensity is not None:
        print(f"Minimum Carbon Intensity: {min_intensity} gCO2/kWh")
        print(f"Maximum Carbon Intensity: {max_intensity} gCO2/kWh")
    else:
        print("No intensity data found in the response.")
else:
    print("No data available")

ConnectionError: HTTPSConnectionPool(host='api.carbonintensity.org.uk', port=443): Max retries exceeded with url: /regional/intensity/2024-06-01/2024-07-13/postcode/DE74 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7f7ae1e70a60>: Failed to resolve 'api.carbonintensity.org.uk' ([Errno -3] Temporary failure in name resolution)"))